## Basic spMspM

This notebook contains some examples of some basic untiled dataflows for matrix multiply (spMspM). Actually, in this code the tensor representation is agnostic as to whether the tensors are dense or sparse (and dense operands can be created below), but for simplicity I just think of them as being sparse.

First, include some libraries

In [ ]:
%run ../prelude.py --style=tree+uncompressed --animation=movie

## Initialize setup

The following cell just creates some sliders to control the creation of the input operand tensors.

In [ ]:
# Initial values

M = 4
N = 4
K = 3
density = [0.9, 0.8]
seed = 10

def set_params(rank_M, rank_N, rank_K, tensor_density, rand_seed):
    global M
    global N
    global K
    global density
    global seed
    
    M = rank_M
    N = rank_N
    K = rank_K
    
    if tensor_density == 'sparse':
        density = [0.9, 0.8]
    elif tensor_density == 'sparser':
        density = [0.9, 0.4]
    else:
        density = [1.0, 1.0]
        
    seed = rand_seed

interactive(set_params,
            rank_M=widgets.IntSlider(min=2, max=10, step=1, value=M),
            rank_N=widgets.IntSlider(min=2, max=10, step=1, value=N),
            rank_K=widgets.IntSlider(min=2, max=10, step=1, value=K),
            tensor_density=['sparse', 'sparser', 'dense'],
            rand_seed=widgets.IntSlider(min=0, max=100, step=1, value=seed))

## Create Input Tensors

Note, the tensor names are suffixed with an ordered list of their rank names.

In [ ]:
a_MK = Tensor.fromRandom(["M", "K"], [M, K], density, 5, seed=seed)
a_MK.setColor("blue").setName("a_MK")
displayTensor(a_MK)

# Create swapped rank version of a
a_KM = a_MK.swapRanks()
a_KM.setName("a_KM")
displayTensor(a_KM)


b_NK = Tensor.fromRandom(["N", "K"], [N, K], density, 5, seed=2*seed)
b_NK.setColor("green").setName("b_NK")
displayTensor(b_NK)

# Create swapped rank version of b
b_KN = b_NK.swapRanks()
b_KN.setName("b_KN")
displayTensor(b_KN)



## Output Stationary/Inner Product

In [ ]:
z_MN = Tensor(rank_ids=["M", "N"], shape=[M, N])
z_MN.setName("z_MN")

a_m = a_MK.getRoot()
b_n = b_NK.getRoot()
z_m = z_MN.getRoot()

canvas = createCanvas(a_MK, b_NK, z_MN)

for m, (z_n, a_k) in z_m << a_m:
    for n, (z_ref, b_k) in z_n << b_n:
        for k, (a_val, b_val) in a_k & b_k:
            z_ref += a_val * b_val
            addFrame(canvas, (m, k), (n, k), (m, n))

displayTensor(z_MN)
displayCanvas(canvas)

z_MN_check = z_MN

## A stationary - row major/Gustavson

In [ ]:
z_MN = Tensor(rank_ids=["M", "N"], shape=[M, N])
z_MN.setName("z_MN")

a_m = a_MK.getRoot()
b_k = b_KN.getRoot()
z_m = z_MN.getRoot()

canvas = createCanvas(a_MK, b_KN, z_MN)

for m, (z_n, a_k) in z_m << a_m:
    for k, (a_val, b_n) in a_k & b_k:
        for n, (z_ref, b_val) in z_n << b_n:
            z_ref += a_val * b_val
            addFrame(canvas, (m, k), (k, n), (m, n))

displayTensor(z_MN)
displayCanvas(canvas)

In [ ]:
z_MN.getRoot() == z_MN_check.getRoot()

## A stationary - column major/Outer Product

In [ ]:
z_MN = Tensor(rank_ids=["M", "N"], shape=[M, N])
z_MN.setName("z_MN")

a_k = a_KM.getRoot()
b_k = b_KN.getRoot()
z_m = z_MN.getRoot()


canvas = createCanvas(a_KM, b_KN, z_MN)

for k, (a_m, b_n) in a_k & b_k:
    for m, (z_n, a_val) in z_m << a_m:
        for n, (z_ref, b_val) in z_n << b_n:
            z_ref += a_val * b_val
            addFrame(canvas, (k, m), (k, n), (m, n))

displayTensor(z_MN)
displayCanvas(canvas)

In [ ]:
z_MN.getRoot() == z_MN_check.getRoot()